In [358]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import time
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline
import re
import json

In [272]:
#set up connection to our db and create our cursor
conn = sqlite3.connect('movies.db')
c = conn.cursor()

In [381]:
#creates initial soup using rotten tomatoes url
def create_soup(year):
    rotten_tomatoes = requests.get("https://www.rottentomatoes.com/top/bestofrt/?year=" + str(year))
    soup = BeautifulSoup(rotten_tomatoes.content, 'html.parser')
    return soup

create_soup(2017)

<!DOCTYPE html>

<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">
<script src="//cdn.optimizely.com/js/594670329.js"></script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="VPPXtECgUUeuATBacnqnCm4ydGO99reF-xgNklSbNbc" name="google-site-verification">
<meta content="034F16304017CA7DCF45D43850915323" name="msvalidate.01"/>
<link href="https://staticv2-4.rottentomatoes.com/static/images/iphone/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="https://staticv2-4.rottentomatoes.com/static/images/icons/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://staticv2-4.rottentomatoes.com/static/styles/css/rt_main.css" rel="stylesheet"/>
<script id="jsonLdSchema" type="application/ld+json">{"@context":"http

In [300]:
#function to create a list of dictionaries containing values from rotten tomatoes page
def find_values_range(year_1, year_2):
    final_results = []
    #iterate through years in range given
    for i in range(year_1, year_2 + 1):
        #use create soup function and pass in the current year (i)
        soup = create_soup(i)
        #our data needed lies within the table class
        values = soup.find_all(class_ = 'table')
        #create list of classes that we will pull to obtain rank, year, score and reviews
        classes = ['bold', 'right hidden-xs', 'tMeterScore', 'unstyled articleLink']
        #create empty lists to store our data
        ranks = []
        names = []
        rotten_scores = []
        ratings = []
        
        #iterate through tags within the 'table' class
        for v in values:
            #for each tag in our classes list, find the tags for each class
            for c in classes:
                find = v.find_all(class_ = c)
                #based on the current c, get text and append to the proper list
                for i in find:
                    if c == 'bold':
                        rank = i.get_text()
                        ranks.append(rank)
                    if c == 'right hidden-xs':
                        rating = i.get_text()
                        #check len of the rating, we need to get rid of the first item because its not a number
                        if len(rating) <= 4:
                            ratings.append(rating)
                    if c == 'tMeterScore':
                        #strip off the crap at the end of the score
                        score = i.get_text().strip('\xa0')
                        rotten_scores.append(score)
                    if c == 'unstyled articleLink':
                        #strip off the crap at the beginning of the name
                        name = i.get_text().strip('\n            ')
                        names.append(name)
        
        #create new dict for each set of movie values
        for i in range(len(ranks)):
            new_dict = {}
            new_dict['rank'] = ranks[i]
            new_dict['name'] = names[i]
            new_dict['rotten_score'] = rotten_scores[i]
            new_dict['ratings'] = ratings[i]
            #append each dictionary to the final_results list
            final_results.append(new_dict)

    return final_results

In [340]:
#create variable in order to make list of just the movie names
dict_ = find_values_range(2009,2018)

In [348]:
#create a list of names from the returned dict
def take_names(dictionary):
    names = []
    for i in range(len(dictionary)):
        names.append(dictionary[i]['name'])
    return names

In [349]:
#strip off the years from the names so that we can use the names to search through the api
def strip_years(dictionary):
    years = take_names(dictionary)
    stripped_names = []
    for year in years:
        year = year[:-7]
        stripped_names.append(year)
    return stripped_names
        

In [350]:
strip_years(dict_)[:10]

['Up',
 'The Hurt Locker',
 'Star Trek',
 'Anvil! The Story of Anvil',
 'Afghan Star',
 'An Education',
 'Drag Me to Hell',
 'District 9',
 'Up in the Air',
 'Fantastic Mr. Fox']

In [273]:
#create table for the rotten tomatoes scraped data
c.execute("""CREATE TABLE IF NOT EXISTS Rotten_Data(rank REAL, name TEXT, rotten_score NUMERIC, Num_of_Reviews INTEGER)""")

In [274]:
#create function to enter the scraped data into the db
def dynamic_data_entry(query):

    for i in query:
        c.execute("INSERT INTO Rotten_Data (Rank, Name,Rotten_Score,Num_of_Reviews) VALUES (?, ?, ?, ?)",
          (i['rank'], i['name'], i['rotten_score'], i['ratings']))

    conn.commit()

dynamic_data_entry(find_values_range(2009,2018))

In [276]:
#view db
c.execute("""SELECT * FROM Rotten_Data""").fetchall()

[(1.0, 'Up (2009)', '98%', 290),
 (2.0, 'The Hurt Locker (2009)', '97%', 279),
 (3.0, 'Star Trek (2009)', '94%', 342),
 (4.0, 'Anvil! The Story of Anvil (2009)', '98%', 135),
 (5.0, 'Afghan Star (2009)', '100%', 60),
 (6.0, 'An Education (2009)', '95%', 189),
 (7.0, 'Drag Me to Hell (2009)', '92%', 262),
 (8.0, 'District 9 (2009)', '90%', 308),
 (9.0, 'Up in the Air (2009)', '91%', 278),
 (10.0, 'Fantastic Mr. Fox (2009)', '92%', 233),
 (11.0, 'In the Loop (2009)', '94%', 172),
 (12.0, 'The Cove (2009)', '95%', 128),
 (13.0, 'Avaze gonjeshk-ha (The Song of Sparrows) (2009)', '98%', 40),
 (14.0, 'Coraline (2009)', '90%', 261),
 (15.0, 'Sweetgrass (2009)', '97%', 60),
 (16.0, 'Inglourious Basterds (2009)', '88%', 317),
 (17.0, 'Precious: Based on the Novel Push by Sapphire (2009)', '91%', 230),
 (18.0, 'Food, Inc. (2009)', '95%', 111),
 (19.0, 'Revanche (2009)', '96%', 81),
 (20.0, 'Zombieland (2009)', '90%', 244),
 (21.0, 'Tulpan (2009)', '96%', 69),
 (22.0,
  'Most Dangerous Man in Ame

In [303]:
#place db into pandas dataframe
rotten_df = pd.read_sql_query("SELECT * FROM Rotten_Data", conn)
rotten_df.head()

,rank,name,rotten_score,Num_of_Reviews
0,1.0,Up (2009),98%,290
1,2.0,The Hurt Locker (2009),97%,279
2,3.0,Star Trek (2009),94%,342
3,4.0,Anvil! The Story of Anvil (2009),98%,135
4,5.0,Afghan Star (2009),100%,60


In [307]:
imdb_top250 = requests.get("https://www.imdb.com/chart/top?ref_=nv_mv_250")
content = imdb_top250.content
soup = BeautifulSoup(content, 'html.parser')

imdb_id=[]
for i in soup.find_all(class_='titleColumn'):
    imdb_id.append(i.find('a').get('href')[7:16])

imdb_id[:5]

['tt0111161', 'tt0068646', 'tt0071562', 'tt0468569', 'tt0050083']

In [515]:
def get_movies(id_list):
    #api_key = '267d993c4e072d2b2046e811f8a45d72'
    movie_list = []

    for i in id_list:
        url = 'https://api.themoviedb.org/3/movie/{}?api_key=267d993c4e072d2b2046e811f8a45d72&language=en-US'.format(i)
        response = requests.get(url)
        movie_list.append(response.json())
        time.sleep(0.5)
    return movie_list

In [314]:
get_movies()

[{'adult': False,
  'backdrop_path': '/j9XKiZrVeViAixVRzCta7h1VU9W.jpg',
  'belongs_to_collection': None,
  'budget': 25000000,
  'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}],
  'homepage': None,
  'id': 278,
  'imdb_id': 'tt0111161',
  'original_language': 'en',
  'original_title': 'The Shawshank Redemption',
  'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.',
  'popularity': 32.926,
  'poster_path': '/9O7gLzmreU0nGkIB6K3BsJbzvNv.jpg',
  'production_companies': [{'id': 97,
    'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png',
    'name': 'Castle Rock Entertainment',
    'origin_country': 'US'},
   {'id': 174,
    '

In [355]:
#create new list and use regular expressions to stip all characters within parenthesis
final_list=[]
for i in range(len(strip_years(dict_))):
    final_list.append(re.sub("[\(\[].*?[\)\]]", "", strip_years(dict_)[i]).strip())

In [357]:
#my api key = a206ee00
def get_movies2():
    movie_list = []

    for i in final_list:
        try:
            url = 'http://www.omdbapi.com/?t={}&plot=full&apikey=a206ee00'.format(i)
            response = requests.get(url)
            movie_list.append(response.json())
            time.sleep(0.1)
        except:
            pass
    return movie_list

In [360]:
with open('omdb_movies', 'w') as fout:
    json.dump(get_movies2(), fout)

In [444]:
with open('omdb_movies') as data:
    d = json.load(data)
rt_df = pd.DataFrame(d)
rt_df.head()

,Actors,Awards,BoxOffice,Country,DVD,Director,Error,Genre,Language,Metascore,...,Runtime,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons
0,"Edward Asner, Christopher Plummer, Jordan Naga...",Won 2 Oscars. Another 74 wins & 81 nominations.,"$292,979,556",USA,10 Nov 2009,"Pete Docter, Bob Peterson(co-director)",NaN,"Animation, Adventure, Comedy, Family",English,88,...,96 min,Up,movie,http://Disney.com/UP,"Pete Docter (story by), Bob Peterson (story by...",2009,tt1049413,8.3,"832,494",NaN
1,"Jeremy Renner, Anthony Mackie, Brian Geraghty,...",Won 6 Oscars. Another 117 wins & 126 nominations.,"$15,700,000",USA,12 Jan 2010,Kathryn Bigelow,NaN,"Drama, Thriller, War","English, Arabic",94,...,131 min,The Hurt Locker,movie,http://www.thehurtlocker-movie.com/,Mark Boal,2008,tt0887912,7.6,"388,856",NaN
2,"Chris Pine, Zachary Quinto, Leonard Nimoy, Eri...",Won 1 Oscar. Another 23 wins & 92 nominations.,"$257,704,099","USA, Germany",17 Nov 2009,J.J. Abrams,NaN,"Action, Adventure, Sci-Fi",English,82,...,127 min,Star Trek,movie,http://www.startrekmovie.com/,"Roberto Orci, Alex Kurtzman, Gene Roddenberry ...",2009,tt0796366,8.0,"552,796",NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Movie not found!,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Habib Amiri, Setara Hussainzada, Rafi Naabzada...",2 wins & 3 nominations.,N/A,UK,30 Mar 2010,Havana Marking,NaN,"Documentary, Music","English, Pushto, Dari",78,...,87 min,Afghan Star,movie,http://www.afghanstardocumentary.com/,N/A,2009,tt1334510,7.3,887,NaN


In [447]:
final_RT_DF = pd.merge(rt_df, rotten_df, right_index = True, left_index = True)
final_RT_DF

,Actors,Awards,BoxOffice,Country,DVD,Director,Error,Genre,Language,Metascore,...,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons,rank,name,rotten_score,Num_of_Reviews
0,"Edward Asner, Christopher Plummer, Jordan Naga...",Won 2 Oscars. Another 74 wins & 81 nominations.,"$292,979,556",USA,10 Nov 2009,"Pete Docter, Bob Peterson(co-director)",NaN,"Animation, Adventure, Comedy, Family",English,88,...,"Pete Docter (story by), Bob Peterson (story by...",2009,tt1049413,8.3,"832,494",NaN,1.0,Up (2009),98%,290
1,"Jeremy Renner, Anthony Mackie, Brian Geraghty,...",Won 6 Oscars. Another 117 wins & 126 nominations.,"$15,700,000",USA,12 Jan 2010,Kathryn Bigelow,NaN,"Drama, Thriller, War","English, Arabic",94,...,Mark Boal,2008,tt0887912,7.6,"388,856",NaN,2.0,The Hurt Locker (2009),97%,279
2,"Chris Pine, Zachary Quinto, Leonard Nimoy, Eri...",Won 1 Oscar. Another 23 wins & 92 nominations.,"$257,704,099","USA, Germany",17 Nov 2009,J.J. Abrams,NaN,"Action, Adventure, Sci-Fi",English,82,...,"Roberto Orci, Alex Kurtzman, Gene Roddenberry ...",2009,tt0796366,8.0,"552,796",NaN,3.0,Star Trek (2009),94%,342
3,NaN,NaN,NaN,NaN,NaN,NaN,Movie not found!,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Anvil! The Story of Anvil (2009),98%,135
4,"Habib Amiri, Setara Hussainzada, Rafi Naabzada...",2 wins & 3 nominations.,N/A,UK,30 Mar 2010,Havana Marking,NaN,"Documentary, Music","English, Pushto, Dari",78,...,N/A,2009,tt1334510,7.3,887,NaN,5.0,Afghan Star (2009),100%,60
5,"Carey Mulligan, Olivia Williams, Alfred Molina...",Nominated for 3 Oscars. Another 35 wins & 91 n...,"$12,512,069","UK, USA",30 Mar 2010,Lone Scherfig,NaN,Drama,"English, French",85,...,"Lynn Barber (memoir), Nick Hornby (screenplay)",2009,tt1174732,7.3,"122,460",NaN,6.0,An Education (2009),95%,189
6,"Alison Lohman, Justin Long, Lorna Raver, Dilee...",7 wins & 22 nominations.,"$42,100,000",USA,13 Oct 2009,Sam Raimi,NaN,"Horror, Thriller","English, Spanish, Hungarian, Czech",83,...,"Sam Raimi, Ivan Raimi",2009,tt1127180,6.6,"175,937",NaN,7.0,Drag Me to Hell (2009),92%,262
7,"Sharlto Copley, Jason Cope, Nathalie Boltt, Sy...",Nominated for 4 Oscars. Another 30 wins & 110 ...,"$115,646,235","South Africa, USA, New Zealand, Canada",22 Dec 2009,Neill Blomkamp,NaN,"Action, Sci-Fi, Thriller","English, Nyanja, Afrikaans, Zulu, Xhosa, South...",81,...,"Neill Blomkamp, Terri Tatchell",2009,tt1136608,7.9,"598,691",NaN,8.0,District 9 (2009),90%,308
8,"George Clooney, Vera Farmiga, Anna Kendrick, J...",Nominated for 6 Oscars. Another 76 wins & 158 ...,"$83,775,048",USA,09 Mar 2010,Jason Reitman,NaN,"Drama, Romance",English,83,...,"Walter Kirn (novel), Jason Reitman (screenplay...",2009,tt1193138,7.4,"298,600",NaN,9.0,Up in the Air (2009),91%,278
9,"George Clooney, Meryl Streep, Jason Schwartzma...",Nominated for 2 Oscars. Another 32 wins & 60 n...,"$21,000,000",USA,23 Mar 2010,Wes Anderson,NaN,"Animation, Adventure, Comedy, Crime, Drama, Fa...","English, French",83,...,"Roald Dahl (novel), Wes Anderson (screenplay),...",2009,tt0432283,7.9,"174,744",NaN,10.0,Fantastic Mr. Fox (2009),92%,233


In [448]:
final_RT_DF = final_RT_DF.dropna(subset=['imdbID'])
final_RT_DF = final_RT_DF.set_index('imdbID')


In [449]:
final_RT_DF = final_RT_DF.drop(columns = ['Type', 'totalSeasons', 'Response', 'Error', 'name', 'DVD', 'Plot', 'Poster'])

In [451]:
final_RT_DF = final_RT_DF.drop(columns = ['Website'])

In [455]:
final_RT_DF = final_RT_DF.drop(columns = ['Released', 'rank'])

In [457]:
final_RT_DF = final_RT_DF.drop(columns = ['Ratings'])

In [478]:
final_RT_DF.columns

Index(['Title', 'Year', 'BoxOffice', 'Metascore', 'rotten_score',
       'Num_of_Reviews', 'imdbRating', 'imdbVotes', 'Country', 'Director',
       'Actors', 'Genre', 'Language', 'Production', 'Rated', 'Runtime',
       'Writer', 'Awards'],
      dtype='object')

In [462]:
final_RT_DF = final_RT_DF[['Title', 'Year', 'BoxOffice', 'Metascore', 'rotten_score', 'Num_of_Reviews', 'imdbRating',
                          'imdbVotes', 'Country', 'Director', 'Actors', 'Genre', 'Language', 'Production', 'Rated', 
                          'Runtime', 'Writer', 'Awards']]


In [467]:
final_RT_DF['rotten_score'] = final_RT_DF['rotten_score'].str.replace('%', '')
final_RT_DF['rotten_score'] = final_RT_DF['rotten_score'].astype(int)

In [489]:
final_RT_DF['BoxOffice'] = final_RT_DF['BoxOffice'].replace('N/A','0')
final_RT_DF['BoxOffice'] = final_RT_DF['BoxOffice'].replace('NaN','0')

In [503]:
final_RT_DF['Metascore'] = final_RT_DF['Metascore'].replace('N/A','0')
final_RT_DF['imdbRating'] = final_RT_DF['imdbRating'].replace('N/A','0')
final_RT_DF['imdbVotes'] = final_RT_DF['imdbVotes'].str.replace(',', '')
final_RT_DF['imdbVotes'] = final_RT_DF['imdbVotes'].replace('N/A', '0')

In [504]:
final_RT_DF['BoxOffice'] = final_RT_DF['BoxOffice'].str.replace('$', '')
final_RT_DF['BoxOffice'] = final_RT_DF['BoxOffice'].str.replace(',', '')
#final_RT_DF['BoxOffice'] = final_RT_DF['BoxOffice'].astype(int)
final_RT_DF['Metascore'] = final_RT_DF['Metascore'].astype(int)
final_RT_DF['Num_of_Reviews'] = final_RT_DF['Num_of_Reviews'].astype(int)
final_RT_DF['imdbRating'] = final_RT_DF['imdbRating'].astype(float)
final_RT_DF['imdbVotes'] = final_RT_DF['imdbVotes'].astype(int)

In [517]:
imdb_id_list = list(final_RT_DF.index)
len(imdb_id_list)

966

In [518]:
with open('imdb_movies', 'w') as fout:
    json.dump(get_movies(imdb_id_list), fout)


In [519]:
with open('imdb_movies') as data_2:
    d_2 = json.load(data_2)
rt_df_2 = pd.DataFrame(d_2)
rt_df_2.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,status_code,status_message,tagline,title,video,vote_average,vote_count
0,False,/6fX7NF6IUJCTVssei7Shgl9J6LL.jpg,None,175000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://disney.go.com/disneypictures/up/,14160.0,tt1049413,en,Up,...,96.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,NaN,,Up,False,7.9,11879.0
1,False,/3iPnZOXR9mpcK8RwvAW7b7Axr8v.jpg,None,15000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",http://www.thehurtlocker-movie.com/,12162.0,tt0887912,en,The Hurt Locker,...,131.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,NaN,You'll know when you're in it.,The Hurt Locker,False,7.3,2961.0
2,False,/1XOSh6BFZbQ0xN75m4avqgzClyG.jpg,"{'id': 115575, 'name': 'Star Trek: Alternate R...",150000000.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",None,13475.0,tt0796366,en,Star Trek,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,NaN,The future begins.,Star Trek,False,7.4,6243.0
3,False,/vN3el4dzS41KnfiS9v20GfEudQm.jpg,None,0.0,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",http://www.afghanstardocumentary.com/,24225.0,tt1334510,en,Afghan Star,...,87.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,NaN,,Afghan Star,False,6.7,9.0
4,False,/4PD0Kc4BGYrYrziCwUPHnESSabZ.jpg,None,7500000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.sonyclassics.com/aneducation/,24684.0,tt1174732,en,An Education,...,100.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,NaN,An education isn't always by the book.,An Education,False,7.0,695.0


In [403]:
def create_soup1(year):
    boxoffice_mojo = requests.get("https://www.boxofficemojo.com/yearly/chart/?yr={}&p=.htm".format(year))
    soup=BeautifulSoup(boxoffice_mojo.content, 'html.parser')
    return soup

In [404]:
final_movies=[]
def find_movies_by_year(year_1, year_2):
    import itertools
    
    final_results = []
    for year in range(year_1, year_2 + 1):
        sample_soup=create_soup1(year).find_all('table', attrs={'cellspacing': '1'})[0]
        box_office_list=[]
        
        for movie in sample_soup.find_all('tr')[2:52]:
            temp_list=[]
            for i in movie.find_all('td'):   
                if movie.find_all('td'):
                    temp_list.append(i.get_text())
                    box_office_list.append(temp_list)

        box_office_list=list(box_office_list for box_office_list,_ in itertools.groupby(box_office_list))

        counter=0
        for box in box_office_list:
            movie_dict={}
            movie_dict["rank"]=box_office_list[counter][0]
            movie_dict["title"]=box_office_list[counter][1]
            movie_dict['year']=year
            movie_dict["studio"]=box_office_list[counter][2]
            movie_dict["gross"]=box_office_list[counter][3]
            movie_dict["num_of_theaters"]=box_office_list[counter][4]
            counter+=1
            final_movies.append(movie_dict)

    return final_movies

In [509]:
boxoffice_df = pd.DataFrame(find_movies_by_year(2009,2018))


In [510]:
boxoffice_df

,gross,num_of_theaters,rank,studio,title,year
0,"$749,766,139","3,461",1,Fox,Avatar,2009
1,"$402,111,870","4,293",2,P/DW,Transformers: Revenge of the Fallen,2009
2,"$301,959,197","4,455",3,WB,Harry Potter and the Half-Blood Prince,2009
3,"$296,623,634","4,124",4,Sum.,The Twilight Saga: New Moon,2009
4,"$293,004,164","3,886",5,BV,Up,2009
5,"$277,322,503","3,545",6,WB,The Hangover,2009
6,"$257,730,019","4,053",7,Par.,Star Trek,2009
7,"$255,959,475","3,407",8,WB,The Blind Side,2009
8,"$219,614,612","3,747",9,Fox,Alvin and the Chipmunks: The Squeakquel,2009
9,"$209,028,679","3,626",10,WB,Sherlock Holmes,2009
